### Statistics Development Notebook


In [1]:
%load_ext autoreload
%autoreload 2

import glob
import os
from copy import deepcopy


import liftout
import pandas as pd
import plotly.express as px
import streamlit as st
from autoscript_sdb_microscope_client.structures import AdornedImage
from liftout.structures import (AutoLiftoutState, Lamella, Sample,
                                load_experiment)
from liftout.tools.data import (AutoLiftoutStatistics,
                                calculate_statistics_dataframe,
                                create_history_dataframe)
pd.set_option('display.max_rows', None)
EXPERIMENT_PATH = r"C:\Users\Admin\Github\autoliftout\liftout\log\dry-run-01-2022-11-09.01-30-41PM"
stats = calculate_statistics_dataframe(EXPERIMENT_PATH)

In [20]:
display(stats.ml)

,feature,lamella,stage,step
0,NeedleTip,01-proper-dingo,Liftout,NEEDLE_EB_DETECTION
1,LamellaLeftEdge,01-proper-dingo,Liftout,NEEDLE_EB_DETECTION
2,NeedleTip,01-proper-dingo,Liftout,NEEDLE_EB_DETECTION
3,LamellaLeftEdge,01-proper-dingo,Liftout,NEEDLE_IB_DETECTION
4,LamellaLeftEdge,01-proper-dingo,Liftout,NEEDLE_IB_DETECTION
5,LamellaLeftEdge,01-proper-dingo,Liftout,NEEDLE_IB_DETECTION
6,LamellaLeftEdge,01-proper-dingo,Liftout,NEEDLE_IB_DETECTION
7,LamellaRightEdge,01-proper-dingo,Liftout,NEEDLE_SEVER_LAMELLA
8,LamellaRightEdge,01-proper-dingo,Liftout,NEEDLE_SEVER_LAMELLA
9,LamellaRightEdge,01-proper-dingo,Landing,LAND_LAMELLA_ON_POST


#### Aggregated Statistics

In [3]:
EXPERIMENT_PATHS = [r"C:\Users\Admin\Github\autoliftout\liftout\log\Dm-E11-3Nov22-2022-11-14.01-20-47PM", 
r"C:\Users\Admin\Github\autoliftout\liftout\log\Dm_E11_3Nov2022_-2022-11-15.11-50-17AM"]

# TODO: add experiment name / date to each dataframe? 
# add date to sample...

for path in EXPERIMENT_PATHS:
    
    stats = calculate_statistics_dataframe(path)
    print(stats.name)
    display(stats.sample)
    display(stats.ml)

    display(stats.click)

Dm-E11-3Nov22-2022-11-14.01-20-47PM


,experiment_name,experiment_path,number,petname,path,lamella.x,lamella.y,lamella.z,lamella.r,lamella.t,...,landing.x,landing.y,landing.z,landing.r,landing.t,landing.coordinate_system,landing_selected,current_stage,last_timestamp,history:
0,Dm-E11-3Nov22-2022-11-14.01-20-47PM,c:\users\admin\github\autoliftout\liftout\log\...,1,01-active-jennet,c:\users\admin\github\autoliftout\liftout\log\...,0.002443,-0.002534,0.031571,-2.286332,0.296705,...,-0.003732,-0.002355,0.031806,-2.286332,0.558504,Raw,True,MillTrench,1.668395e+09,1
1,Dm-E11-3Nov22-2022-11-14.01-20-47PM,c:\users\admin\github\autoliftout\liftout\log\...,2,02-native-dory,c:\users\admin\github\autoliftout\liftout\log\...,0.002427,-0.002494,0.031598,-2.286332,0.296705,...,-0.003511,-0.002351,0.031807,-2.286332,0.558504,Raw,True,Finished,1.668414e+09,8
2,Dm-E11-3Nov22-2022-11-14.01-20-47PM,c:\users\admin\github\autoliftout\liftout\log\...,3,03-saved-lemur,c:\users\admin\github\autoliftout\liftout\log\...,0.002385,-0.002408,0.031598,-2.286332,0.296705,...,-0.003734,-0.002355,0.031788,-2.286366,0.558502,Raw,True,Finished,1.668414e+09,8
3,Dm-E11-3Nov22-2022-11-14.01-20-47PM,c:\users\admin\github\autoliftout\liftout\log\...,4,04-kind-toad,c:\users\admin\github\autoliftout\liftout\log\...,0.002378,-0.002341,0.031643,-2.286332,0.296705,...,-0.003511,-0.002344,0.031794,-2.286366,0.558502,Raw,True,Liftout,1.668406e+09,3


,feature,correct,lamella,stage,step
0,NeedleTip,False,04-kind-toad,MillJCut,REF_JCUT
1,ImageCentre,True,04-kind-toad,MillJCut,REF_JCUT
2,NeedleTip,True,04-kind-toad,MillJCut,REF_JCUT
3,ImageCentre,True,04-kind-toad,MillJCut,REF_JCUT
4,NeedleTip,True,04-kind-toad,MillJCut,REF_JCUT
5,ImageCentre,True,04-kind-toad,MillJCut,REF_JCUT
6,NeedleTip,True,04-kind-toad,MillJCut,REF_JCUT
7,ImageCentre,True,04-kind-toad,MillJCut,REF_JCUT
8,NeedleTip,True,04-kind-toad,MillJCut,REF_JCUT
9,ImageCentre,True,04-kind-toad,MillJCut,REF_JCUT


,beam_type,type,x,y,lamella,stage,step
0,ELECTRON,Stable,-1.370000e-04,-1.710000e-04,None,None,None
1,ION,Eucentric,-1.220000e-05,2.980000e-04,None,None,None
2,ION,Eucentric,1.460000e-05,1.070000e-04,None,None,None
3,ION,Eucentric,1.460000e-05,-2.480000e-05,None,None,None
4,ELECTRON,Stable,2.420000e-05,1.300000e-05,None,None,None
5,ION,Eucentric,3.410000e-06,2.530000e-05,None,None,None
6,ION,Stable,7.160000e-05,2.480000e-06,None,None,None
7,ION,Stable,-2.890000e-05,4.780000e-05,None,None,None
8,ION,Stable,1.310000e-05,7.300000e-07,None,None,None
9,ELECTRON,Stable,4.740000e-05,-2.190000e-04,None,None,None


Dm_E11_3Nov2022_-2022-11-15.11-50-17AM


,experiment_name,experiment_path,number,petname,path,lamella.x,lamella.y,lamella.z,lamella.r,lamella.t,...,landing.x,landing.y,landing.z,landing.r,landing.t,landing.coordinate_system,landing_selected,current_stage,last_timestamp,history:
0,Dm_E11_3Nov2022_-2022-11-15.11-50-17AM,c:\users\admin\github\autoliftout\liftout\log\...,1,01-exact-quagga,c:\users\admin\github\autoliftout\liftout\log\...,0.002436,-0.002394,0.031616,-2.286332,0.296707,...,-0.003726,-0.002298,0.031862,-2.286299,0.558502,Raw,True,MillJCut,1.668485e+09,2
1,Dm_E11_3Nov2022_-2022-11-15.11-50-17AM,c:\users\admin\github\autoliftout\liftout\log\...,2,02-caring-rat,c:\users\admin\github\autoliftout\liftout\log\...,0.002406,-0.002341,0.031662,-2.286332,0.296707,...,-0.003507,-0.002304,0.031859,-2.286299,0.558502,Raw,True,Liftout,1.668489e+09,3
2,Dm_E11_3Nov2022_-2022-11-15.11-50-17AM,c:\users\admin\github\autoliftout\liftout\log\...,3,03-tender-salmon,c:\users\admin\github\autoliftout\liftout\log\...,0.002438,-0.002277,0.031705,-2.286332,0.296707,...,-0.003291,-0.002309,0.031856,-2.286299,0.558502,Raw,True,Finished,1.668498e+09,8
3,Dm_E11_3Nov2022_-2022-11-15.11-50-17AM,c:\users\admin\github\autoliftout\liftout\log\...,4,04-cool-monkey,c:\users\admin\github\autoliftout\liftout\log\...,0.002469,-0.002294,0.031694,-2.286332,0.296707,...,-0.003069,-0.002306,0.031858,-2.286299,0.558502,Raw,True,Finished,1.668498e+09,8


,feature,correct,lamella,stage,step
0,LamellaLeftEdge,False,01-exact-quagga,Liftout,NEEDLE_EB_DETECTION
1,NeedleTip,True,01-exact-quagga,Liftout,NEEDLE_EB_DETECTION
2,NeedleTip,False,01-exact-quagga,Liftout,NEEDLE_IB_DETECTION
3,LamellaLeftEdge,False,01-exact-quagga,Liftout,NEEDLE_IB_DETECTION
4,NeedleTip,True,02-caring-rat,Liftout,NEEDLE_EB_DETECTION
5,LamellaLeftEdge,True,02-caring-rat,Liftout,NEEDLE_EB_DETECTION
6,NeedleTip,False,02-caring-rat,Liftout,NEEDLE_IB_DETECTION
7,LamellaLeftEdge,False,02-caring-rat,Liftout,NEEDLE_IB_DETECTION
8,LamellaRightEdge,True,02-caring-rat,Liftout,NEEDLE_SEVER_LAMELLA
9,ImageCentre,True,02-caring-rat,Liftout,NEEDLE_SEVER_LAMELLA


,beam_type,type,x,y,lamella,stage,step
0,ELECTRON,Stable,-1.790000e-04,-2.520000e-04,None,None,None
1,ION,Eucentric,-7.560000e-06,2.840000e-04,None,None,None
2,ION,Eucentric,1.580000e-05,9.900000e-05,None,None,None
3,ION,Eucentric,1.110000e-05,-1.550000e-05,None,None,None
4,ION,Stable,7.560000e-05,3.410000e-05,None,None,None
5,ELECTRON,Stable,-3.020000e-05,3.130000e-05,None,None,None
6,ION,Eucentric,-4.820000e-07,9.100000e-06,None,None,None
7,ION,Stable,3.050000e-05,4.690000e-05,None,None,None
8,ION,Stable,-6.140000e-05,2.140000e-05,None,None,None
9,ELECTRON,Stable,-1.170000e-05,-1.610000e-06,None,None,None
